In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
from tensorflow.keras.models import load_model
# Load the trained model
model = load_model('/content/drive/MyDrive/Save Model/updatedModel1.h5')


In [26]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os

def detect_and_display(img_path, model, train_dir, image_size=128):

    class_labels = sorted(os.listdir(train_dir))  # Ensure label order

    try:
        img = load_img(img_path, target_size=(image_size, image_size), color_mode="rgb")
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        confidence_score = predictions[0][predicted_class_index]

        result = "No Tumor" if class_labels[predicted_class_index] == 'notumor' else f"Tumor: {class_labels[predicted_class_index]}"

        plt.imshow(load_img(img_path))
        plt.axis('off')
        plt.title(f"{result} (Confidence: {confidence_score * 100:.2f}%)")
        plt.show()

    except Exception as e:
        print("Error processing the image:", str(e))

In [27]:
!pip install streamlit -q

In [28]:
!wget -q -O - ipv4.icanhazip.com

34.91.75.15


In [29]:
%%writefile app.py
# Streamlit application code for deployment
import streamlit as st
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image

train_dir = '/content/drive/MyDrive/tumor/Training/'
# Get a sorted list of class labels (ensuring consistent mapping)
class_labels = sorted(os.listdir(train_dir))

# Cache the model so that it loads only once
@st.cache_resource
def load_trained_model():
    model = load_model('/content/drive/MyDrive/Save Model/updatedModel1.h5')
    return model

model = load_trained_model()

# Function to preprocess the image (similar to your open_images and augment functions)
def preprocess_image(image, image_size=128):
    img = image.convert("RGB")
    img = img.resize((image_size, image_size))
    # Convert to numpy array and normalize pixel values to [0,1]
    img_array = img_to_array(img) / 255.0
    # Expand dimensions to create a batch of 1
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Prediction function (correlates with your detect_and_display)
def predict_tumor(image, model, image_size=128):
    img_array = preprocess_image(image, image_size)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    confidence_score = predictions[0][predicted_class_index]

    # Use lower-case check to handle potential case mismatches
    if class_labels[predicted_class_index].lower() == 'notumor':
        result = "No Tumor"
    else:
        result = f"Tumor: {class_labels[predicted_class_index]}"

    return result, confidence_score

# Streamlit app
def main():
    st.title("Tumor Detection Application")
    st.write("Upload an image and the model will predict whether a tumor is present.")

    uploaded_file = st.file_uploader("Choose an image file...", type=["jpg", "jpeg", "png"])
    if uploaded_file is not None:
        # Open the image using PIL and display it
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_container_width=True)

        if st.button("Run Prediction"):
            result, confidence = predict_tumor(image, model)
            confidence_percent = confidence * 100

            st.markdown(f"### Prediction: **{result}**")
            st.markdown(f"### Confidence: **{confidence_percent:.2f}%**")

            # Provide feedback based on prediction and confidence
            if result == "No Tumor":
                if confidence > 0.7:
                    st.success("The model is highly confident that there is no tumor.")
                else:
                    st.warning("The model predicts no tumor, but confidence is moderate.")
            else:
                if confidence > 0.7:
                    st.error("The model predicts a tumor with high confidence!")
                else:
                    st.warning("The model predicts a tumor, but confidence is moderate.")

if __name__ == '__main__':
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.75.15:8501

⠸⠼⠴your url is: https://better-shoes-build.loca.lt
2025-02-21 11:49:54.981628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740138595.029261   11290 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740138595.042766   11290 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 11:50:02.095687: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 